In [1]:
import numpy as np 
import pandas as pd 

In [2]:
class Team:
    def __init__(self, name, wins, games_played=41):
        self.name = name
        self.wins = wins
        self.games_played = games_played
        self.n = wins / games_played

        self.a1, self.b1, self.c1, self.d1, self.a2, self.b2, self.c2, self.d2 = generate_ordered_constants(self.n)

    def payoff_matrix(self):
        playoff_matrix = np.array([[self.a1, self.b1],
                                   [self.c1, self.d1]])
        draft_matrix = np.array([[self.a2, self.b2],
                                 [self.c2, self.d2]])
        return playoff_matrix, draft_matrix

    def expected_payoff_matrix(self):
        playoff_matrix, draft_matrix = self.payoff_matrix()
        return self.n * playoff_matrix + (1 - self.n) * draft_matrix

    def __str__(self):
        mat = self.expected_payoff_matrix()
        return f"{self.name} ({self.wins}-{self.games_played - self.wins}, n={self.n:.2f})\nExpected Payoff Matrix:\n" + \
               f"         Opponent\n         T       W\n" + \
               f"     ----------------\n" + \
               f"T |  {mat[0,0]:.2f}   {mat[0,1]:.2f}\n" + \
               f"W |  {mat[1,0]:.2f}   {mat[1,1]:.2f}"

    def describe_matrix(self):
        mat = self.expected_payoff_matrix()
        print("MAT OUTPUT", mat)
        return f"""
--- {self.name} Game Theory Matrix (n = {self.n:.2f}) ---

You = rows; Opponent = columns

         Opponent
           T       W
       ---------------
 T |  Tank vs Tank   Tank vs Win
 W |  Win vs Tank    Win vs Win

Numerical Expected Payoffs:
Tanker vs Tanker   = {mat[0,0]:.2f}
Tanker vs Winner   = {mat[0,1]:.2f}
Winner vs Tanker   = {mat[1,0]:.2f}
Winner vs Winner   = {mat[1,1]:.2f}

Interpretation:
- Row T: What happens if YOU tank, vs opponent strategy
- Row W: What happens if YOU try to win, vs opponent strategy

Use this to compare which row yields a better average result.
"""



In [3]:
def tank_GT(team: Team):
    n = team.n
    playoff_matrix, draft_matrix = team.payoff_matrix()

    # Expected payoff matrix as weighted average
    expected_matrix = n * playoff_matrix + (1 - n) * draft_matrix

    return expected_matrix


In [4]:
def generate_ordered_constants(n):
    # n: team’s current win rate (between 0 and 1)

    # Draft scenario (1 - n)
    a2 = (1 - n) * 10 + np.random.uniform(0, 1)      # big reward for tanking if doing badly
    b2 = (1 - n) * 9 + np.random.uniform(0, 1)
    c2 = (1 - n) * 6 + np.random.uniform(0, 1)
    d2 = (1 - n) * 2 + np.random.uniform(0, 1)

    # Playoff scenario (n)
    a1 = a2 + np.random.uniform(1, 2)  # still prefer to sneak into playoffs even when tanking
    b1 = b2 - np.random.uniform(2, 3)  # low seed playoff worse than good draft pick
    c1 = c2 + np.random.uniform(2, 3)  # winning when others tank: great
    d1 = c1 + np.random.uniform(1, 2)  # both good → best outcome

    # Enforce strict inequalities
    assert a1 > a2 and b1 < b2 and c1 > c2 and d1 > d2 and c1 < d1

    return a1, b1, c1, d1, a2, b2, c2, d2


In [5]:
teams = [
    Team("Lakers", 21),
    Team("Wizards", 12),
    Team("Celtics", 29),
    Team("Pistons", 8)
]

for team in teams:
    print(f"{team.name} GT Matrix (n = {team.n:.2f}):")
    print(tank_GT(team))
    print()


Lakers GT Matrix (n = 0.51):
[[5.6544451  3.50212257]
 [4.46912099 4.73279705]]

Wizards GT Matrix (n = 0.29):
[[8.41774535 6.05181459]
 [5.77133936 3.6357592 ]]

Celtics GT Matrix (n = 0.71):
[[4.14657905 1.35039316]
 [3.43859609 4.45820059]]

Pistons GT Matrix (n = 0.20):
[[8.92210715 7.31600317]
 [5.69319539 3.39315502]]



In [6]:
team = Team("Rockets", wins=15)
print(team)
print(team.describe_matrix())


Rockets (15-26, n=0.37)
Expected Payoff Matrix:
         Opponent
         T       W
     ----------------
T |  6.94   5.60
W |  5.47   4.58
MAT OUTPUT [[6.94352879 5.6035104 ]
 [5.46590903 4.57815847]]

--- Rockets Game Theory Matrix (n = 0.37) ---

You = rows; Opponent = columns

         Opponent
           T       W
       ---------------
 T |  Tank vs Tank   Tank vs Win
 W |  Win vs Tank    Win vs Win

Numerical Expected Payoffs:
Tanker vs Tanker   = 6.94
Tanker vs Winner   = 5.60
Winner vs Tanker   = 5.47
Winner vs Winner   = 4.58

Interpretation:
- Row T: What happens if YOU tank, vs opponent strategy
- Row W: What happens if YOU try to win, vs opponent strategy

Use this to compare which row yields a better average result.

